In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
ds=xr.open_dataset(dir_data+'precip_datasets_compare_persiann_x_6070_test.nc')

total_list_persiann_ocean=ds['total_list_persiann_ocean'].values.tolist()
freq_list_persiann_ocean=ds['freq_list_persiann_ocean'].values.tolist()
intensity_list_persiann_ocean=ds['intensity_list_persiann_ocean'].values.tolist()

total_list_persiann_land=ds['total_list_persiann_land'].values.tolist()
freq_list_persiann_land=ds['freq_list_persiann_land'].values.tolist()
intensity_list_persiann_land=ds['intensity_list_persiann_land'].values.tolist()

total_persiann_ocean_monthly=ds['total_persiann_ocean_monthly'].values
freq_persiann_ocean_monthly=ds['freq_persiann_ocean_monthly'].values
intensity_persiann_ocean_monthly=ds['intensity_persiann_ocean_monthly'].values

total_persiann_land_monthly=ds['total_persiann_land_monthly'].values
freq_persiann_land_monthly=ds['freq_persiann_land_monthly'].values
intensity_persiann_land_monthly=ds['intensity_persiann_land_monthly'].values


ds=xr.open_dataset(dir_data+'precip_datasets_compare_persiann_x_6070_test_unadj.nc')

total_list_persiann_ocean1=ds['total_list_persiann_ocean'].values.tolist()
freq_list_persiann_ocean1=ds['freq_list_persiann_ocean'].values.tolist()
intensity_list_persiann_ocean1=ds['intensity_list_persiann_ocean'].values.tolist()

total_list_persiann_land1=ds['total_list_persiann_land'].values.tolist()
freq_list_persiann_land1=ds['freq_list_persiann_land'].values.tolist()
intensity_list_persiann_land1=ds['intensity_list_persiann_land'].values.tolist()

total_persiann_ocean_monthly1=ds['total_persiann_ocean_monthly'].values
freq_persiann_ocean_monthly1=ds['freq_persiann_ocean_monthly'].values
intensity_persiann_ocean_monthly1=ds['intensity_persiann_ocean_monthly'].values

total_persiann_land_monthly1=ds['total_persiann_land_monthly'].values
freq_persiann_land_monthly1=ds['freq_persiann_land_monthly'].values
intensity_persiann_land_monthly1=ds['intensity_persiann_land_monthly'].values

In [ ]:
month_days=[31,28.25,31,30,31,30,31,31,30,31,30,31]

yrs=np.arange(1983,2023,1)
for i in range(len(yrs)):
    if i==0:
        total_month_persiann_ocean=list(total_persiann_ocean_monthly[:,i])
        total_month_persiann_land=list(total_persiann_land_monthly[:,i])
        total_month_persiann_ocean1=list(total_persiann_ocean_monthly1[:,i])
        total_month_persiann_land1=list(total_persiann_land_monthly1[:,i])
        
    else:
        
        total_month_persiann_ocean=total_month_persiann_ocean+list(total_persiann_ocean_monthly[:,i])
        total_month_persiann_land=total_month_persiann_land+list(total_persiann_land_monthly[:,i])
        total_month_persiann_ocean1=total_month_persiann_ocean1+list(total_persiann_ocean_monthly1[:,i])
        total_month_persiann_land1=total_month_persiann_land1+list(total_persiann_land_monthly1[:,i])

x_arr=np.arange(1982.5,2022.5,1./12.)

test=[x/12. for x in total_list_persiann_ocean]

zipped_years=list(zip(yrs,test))
select_years=[x[0] for x in zipped_years if x[1]>=0]
select_values=[x[1] for x in zipped_years if x[1]>=0]
r=st.linregress(select_years,select_values)

change=r[0]*len(yrs)
mean=float(sum(select_values))/float(len(select_values))
label_5='Adjusted ocean (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

if 1==1:
    test=[x/12. for x in total_list_persiann_land]
    
    zipped_years=list(zip(yrs,test))
    select_years=[x[0] for x in zipped_years if x[1]>=0]
    select_values=[x[1] for x in zipped_years if x[1]>=0]
    r=st.linregress(select_years,select_values)

    change=r[0]*len(yrs)
    mean=float(sum(select_values))/float(len(select_values))
    label_6='Adjusted land (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

    test=[x/12. for x in total_list_persiann_ocean1]
    print(test)
    r=st.linregress(yrs,test)
    change=r[0]*len(yrs)
    mean=float(sum(test))/float(len(test))
    label_7='Unadjusted ocean (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

    test=[x/12. for x in total_list_persiann_land1]
    r=st.linregress(yrs,test)
    change=r[0]*len(yrs)
    mean=float(sum(test))/float(len(test))
    label_8='Unadjusted land (Mean='+str(round(mean,1))+', '+'$\Delta$='+str(round(change,2))+', p='+str(round(r[3],3))+')'

fig=plt.figure(figsize=(16,8))
ax=plt.subplot(1,1,1)

ax.plot(yrs,[x/12. for x in total_list_persiann_ocean],color='b',linewidth=5,label=label_5)
ax.plot(x_arr,total_month_persiann_ocean,color='b',linewidth=1,alpha=0.75)

if 1==1:
    ax.plot(yrs,[x/12. for x in total_list_persiann_land],color='orange',linewidth=5,label=label_6)
    ax.plot(x_arr,total_month_persiann_land,color='orange',linewidth=1,alpha=0.75)

    ax.plot(yrs,[x/12. for x in total_list_persiann_ocean1],color='limegreen',linewidth=5,label=label_7)
    ax.plot(x_arr,total_month_persiann_ocean1,color='limegreen',linewidth=1,alpha=0.75)

    ax.plot(yrs,[x/12. for x in total_list_persiann_land1],color='m',linewidth=5,label=label_8)
    ax.plot(x_arr,total_month_persiann_land1,color='m',linewidth=1,alpha=0.75)

#plt.axvline(x=1990,linewidth=3,linestyle='--',color='k')
#plt.axvline(x=2021,linewidth=3,linestyle='--',color='k')

ax.tick_params(labelsize=18)
ax.set_xlabel('Year',fontsize=20)
ax.set_ylabel('mm month$^{-1}$',fontsize=20)
#ax.set_title(r'$\bf{Wet}$'+' '+r'$\bf{day}$'+' '+r'$\bf{total}$',fontsize=32,pad=5)
ax.set_title(r'$\bf{R98p_{all},}$'+' '+r'$\bf{PERSIANN-CDR}$',fontsize=29,pad=5)
ax.grid(True)
plt.legend(loc='upper center',fontsize=15.8,ncol=2)
#ax.set_xlim(1978.8,2022.2)
ax.set_xlim(1982.3,2022.6)
ax.set_ylim(0,38)
for axis in ['top', 'bottom', 'left', 'right']:
    ax.spines[axis].set_linewidth(1.75)
#ax.fill_between(np.arange(1978,1991,1),200,1150,color='gray',alpha=0.3)
#ax.fill_between(np.arange(2021,2024,1),200,1150,color='gray',alpha=0.3)
plt.show()

In [ ]:
fig.savefig(dir_data+'adj_illust_1.png')